In [ ]:
!pip install dfply
!pip install textattack
!pip install sacremoses
!pip install textaugment
!pip install nlpaug
!pip install spacy
!python -m spacy download en_core_web_md 

In [ ]:
import spacy
from nltk.util import ngrams,bigrams,trigrams
import numpy as np
import pandas as pd
from dfply import *
import os
import string
from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter
import nlpaug.augmenter.word as naw
from textaugment import EDA
nlp_md = spacy.load("en_core_web_md")
backtrans_aug = naw.BackTranslationAug()
random_aug    = EDA()
wordnet_aug   = WordNetAugmenter()
embed_aug     = EmbeddingAugmenter()
eda_aug       = EasyDataAugmenter()

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/315k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/849k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315k [00:00<?, ?B/s]

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:32<00:00, 14.6MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpsb2a7x3e.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/AMAZON_FINAL.xlsx',engine='openpyxl',sheet_name = "DATA")
pr = data >> mask(X.FILTER=='No') 
pr.reset_index(drop=True, inplace=True)

In [ ]:
au = pr >> mask(X.ASPECT.isin(['Alarm','Calories Burned','Clock Face','Connect','Distance Tracking','Mobile App','Notifications','Price','Reminders','Swimproof + Swim tracking','Syncing','Watch Face','Waterproof']))
au.reset_index(drop=True, inplace=True)

In [ ]:
def aumented(device, rate, country, comments, clause, aspect, sentiment, filter):
  result = np.column_stack((np.repeat(device,6,axis=0),
                            np.repeat(rate,6,axis=0),
                            np.repeat(country,6,axis=0),
                            np.repeat(comments,6,axis=0),
                            np.repeat(clause,6,axis=0),
                            np.repeat(aspect,6,axis=0),
                            np.repeat(sentiment,6,axis=0),
                            np.repeat(filter,6,axis=0),
                            [wordnet_aug.augment(clause)[0],embed_aug.augment(clause)[0],eda_aug.augment(clause)[0],eda_aug.augment(clause)[1],backtrans_aug.augment(clause)[0],random_aug.random_insertion(clause)]))
  result = pd.DataFrame(result, columns = ['DEVICE','RATE','COUNTRY','COMMENTS','CLAUSE','ASPECT','SENTIMENT','FILTER','AUGMENTED']) >> mutate(AUGMENTED = X.AUGMENTED.str.lower())
  result = result[result.AUGMENTED != result.CLAUSE].reset_index()
  return result

In [ ]:
def preprocesar_light(sentence):
  nounL = []
  doc = nlp_md(sentence.lower().strip())
  for n in doc:
    nounL.append(n.lemma_)
  return tuple(nounL)

def get_sentence_nounverb_aspect(unigram,bigram,trigram,aspects):
  fa = []#set()
  ngrams = unigram+bigram+trigram
  for t in ngrams:
    if t in aspects:
      fa.append(t)
  return fa

def get_aspect(sentence,aspect_lema):
  preprocess_light = preprocesar_light(sentence)
  unigram = list(ngrams(preprocess_light,1))
  bigram = list(ngrams(preprocess_light,2))
  trigram = list(ngrams(preprocess_light,3))
  everyaspect = set(aspect_lema)
  foundedaspect = get_sentence_nounverb_aspect(unigram,bigram,trigram,everyaspect)
  return foundedaspect

def get_specific_aspects(sentence):
    if sentence == 'alarm':
      return np.array('Alarm')
    elif sentence == 'mobile' or sentence == 'mobile app':
      return np.array('Mobile App')  
    elif sentence == 'screen':
      return np.array('Screen')    
    elif sentence == 'calorie':
      return np.array('Calories Burned')
    elif sentence == 'clock':
      return np.array('Clock Face')
    elif sentence == 'connect':
      return np.array('Connect')
    elif sentence == 'distance':
      return np.array('Distance Tracking')
    elif sentence == 'heart':
      return np.array('Heart Rate Tracking')
    elif sentence == 'notification':
      return np.array('Notifications')
    elif sentence == 'price':
      return np.array('Price')
    elif sentence == 'reminder':
      return np.array('Reminders')
    elif sentence == 'swim':
      return np.array('Swimproof + Swim tracking')
    elif sentence == 'sync':
      return np.array('Syncing')
    elif sentence == 'watch':
      return np.array('Watch Face')
    elif sentence == 'step':
      return np.array('Step Counter')
    elif sentence == 'battery':
      return np.array('Battery Life')
    elif sentence == 'sleep':
      return np.array('Sleep Tracker')
    elif sentence == 'app':
      return np.array('App')
    elif sentence == 'waterproof' or sentence == 'water resistance':
      return np.array('Waterproof')
    else: 
      return np.array('None')

In [ ]:
result = []
for i in np.arange(0, len(au), 1):
  result.append(aumented(au.DEVICE[i],au.RATE[i],au.COUNTRY[i],au.COMMENTS[i],au.CLAUSE[i],au.ASPECT[i],au.SENTIMENT[i],au.FILTER[i]))
final = pd.concat(result, axis=0)

In [ ]:
NOUNS = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NOUNS.xlsx',engine='openpyxl',sheet_name = "DATA")
NOUNS["aspect"] = NOUNS.aspect.str.replace("\xa0", " ")
NOUNS["aspect_lema"] = NOUNS["aspect"].apply(preprocesar_light)

In [ ]:
final = final[~final["AUGMENTED"].isna()].reset_index(drop=True)
FIL = []
for i in range(0,final.shape[0],1): FIL.append(get_aspect(final.AUGMENTED[i],NOUNS.aspect_lema))
final["ASPECT_2"] = pd.Series(FIL)
final["LEN_TOK"] = final["ASPECT_2"].apply(len)
final.loc[final["LEN_TOK"] > 0,"ASPECT_2"] = [get_specific_aspects(item[0][0]) for item in final.loc[final["LEN_TOK"] > 0,"ASPECT_2"]]
final["ASPECT_2"] = final["ASPECT_2"].astype(str).replace('[]','None')
final = final[final.ASPECT == final.ASPECT_2].reset_index() >> select(~X.ASPECT_2, ~X.LEN_TOK, ~X.level_0)
del final['index']

In [ ]:
aumented = pd.concat([
final >> mask(X.ASPECT == 'Alarm')                      >> sample(n = 474),
final >> mask(X.ASPECT == 'Calories Burned')            >> sample(n = 395),
final >> mask(X.ASPECT == 'Clock Face')                 ,
final >> mask(X.ASPECT == 'Connect')                    >> sample(n = 306),
final >> mask(X.ASPECT == 'Distance Tracking')          >> sample(n = 480),
final >> mask(X.ASPECT == 'Mobile App')                 >> sample(n = 487),
final >> mask(X.ASPECT == 'Notifications')              >> sample(n = 317),
final >> mask(X.ASPECT == 'Price')                      >> sample(n = 181),
final >> mask(X.ASPECT == 'Reminders')                  >> sample(n = 540),
final >> mask(X.ASPECT == 'Swimproof + Swim tracking')  >> sample(n = 462),
final >> mask(X.ASPECT == 'Syncing')                    >> sample(n = 194),
final >> mask(X.ASPECT == 'Watch Face')                 >> sample(n = 566),
final >> mask(X.ASPECT == 'Waterproof')                 >> sample(n = 437)
], axis=0) >> select(X.DEVICE, X.RATE, X.COUNTRY, X.COMMENTS, X.AUGMENTED, X.ASPECT, X.SENTIMENT, X.FILTER) >> rename(CLAUSE = X.AUGMENTED) >> mutate(AUGMENTED = 'Si') 

In [ ]:
training = data >> mask(X.FILTER=='No') 
training = pd.concat([
training >> mask(X.ASPECT == 'App')                   >> sample(n = 658),
training >> mask(X.ASPECT == 'Battery Life')          >> sample(n = 658),
training >> mask(X.ASPECT == 'Heart Rate Tracking')   >> sample(n = 658),
training >> mask(X.ASPECT == 'None')                  >> sample(n = 658),
training >> mask(X.ASPECT == 'Screen')                >> sample(n = 658),
training >> mask(X.ASPECT == 'Sleep Tracker')         >> sample(n = 658),
training >> mask(X.ASPECT == 'Step Counter')          >> sample(n = 658),
training >> mask(X.ASPECT == 'Alarm'),
training >> mask(X.ASPECT == 'Calories Burned'),
training >> mask(X.ASPECT == 'Clock Face'),
training >> mask(X.ASPECT == 'Connect'),
training >> mask(X.ASPECT == 'Distance Tracking'),
training >> mask(X.ASPECT == 'Mobile App'),
training >> mask(X.ASPECT == 'Notifications'),
training >> mask(X.ASPECT == 'Price'),
training >> mask(X.ASPECT == 'Reminders'),
training >> mask(X.ASPECT == 'Swimproof + Swim tracking'),
training >> mask(X.ASPECT == 'Syncing'),                    
training >> mask(X.ASPECT == 'Watch Face'),               
training >> mask(X.ASPECT == 'Waterproof')                
], axis=0) >> mutate(AUGMENTED = 'No') 

In [ ]:
pd.concat([training , aumented]) >> group_by('ASPECT') >> summarize(Cuenta = X.ASPECT.value_counts())

In [ ]:
testing = data >> mask(X.FILTER=='Si') >> mutate(AUGMENTED = 'No') 
aumented_final = pd.concat([training, aumented, testing])
aumented_final.to_excel('/content/drive/My Drive/Colab Notebooks/AMAZON_AUGMENTED.xlsx', index = None, header=True, sheet_name = "DATA")